In [ ]:
%%shell
# 1. Linux paketlerini güncelle
sudo apt -y update

# 2. Google Chrome'un en güncel ve kararlı sürümünü indirip kur (Çökme sorununu çözer)
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
sudo apt install ./google-chrome-stable_current_amd64.deb -y

# 3. Gerekli Python kütüphanelerini kur
pip install selenium webdriver-manager pandas

In [ ]:
import json
import pandas as pd
from bs4 import BeautifulSoup
import re

# 1. ADIM: Dosya Yükleme ve Okuma

yuklenen_dosya_bilgisi = files.upload() # Yüklediğiniz dosyanın adı
dosya_adi = list(yuklenen_dosya_bilgisi.keys())[0] # Yüklenen dosyanın adını al

try:
    with open(dosya_adi, 'r', encoding='utf-8') as f:
        html_content = f.read()
    print(f"'{dosya_adi}' dosyası okundu. ({len(html_content)} karakter)")
except UnicodeDecodeError:
    # UTF-8 hata verirse alternatif okuma
    with open(dosya_adi, 'r', encoding='latin-1') as f:
        html_content = f.read()
    print(f"'{dosya_adi}' dosyası (Latin-1) okundu.")

soup = BeautifulSoup(html_content, 'html.parser')
urunler_listesi = []

# ---------------------------------------------------------
# YÖNTEM A: JSON VERİSİNİ ÇEKME (En Temiz Yöntem)
# ---------------------------------------------------------
# Watsons genellikle verileri <script id="__NEXT_DATA__"> içinde tutar.
print("\nYöntem A (JSON) deneniyor...")
script_tag = soup.find("script", {"id": "__NEXT_DATA__"})

if script_tag:
    try:
        data = json.loads(script_tag.string)
        # Genellikle veri şu yolda bulunur: props -> pageProps -> initialState -> products -> list
        # Bu yapı sayfadan sayfaya değişebilir, bu yüzden güvenli bir arama yapıyoruz.

        # JSON içindeki tüm ürün listelerini bulmaya çalışan yardımcı fonksiyon
        def find_products_in_json(obj, found_items):
            if isinstance(obj, dict):
                if 'code' in obj and 'name' in obj and 'price' in obj: # Ürün imzası
                    found_items.append(obj)
                for k, v in obj.items():
                    find_products_in_json(v, found_items)
            elif isinstance(obj, list):
                for item in obj:
                    find_products_in_json(item, found_items)

        raw_products = []
        find_products_in_json(data, raw_products)

        if raw_products:
            print(f"✅ JSON içinde {len(raw_products)} adet potansiyel ürün bulundu.")

            for item in raw_products:
                # Fiyat verisini bulma (yapı değişebilir)
                fiyat = "0"
                if 'price' in item:
                    if isinstance(item['price'], dict):
                        fiyat = item['price'].get('formattedValue', item['price'].get('value', '0'))
                    else:
                        fiyat = str(item['price'])

                # Resim
                resim = "Yok"
                if 'images' in item and item['images']:
                     resim = item['images'][0].get('url', 'Yok')
                elif 'image' in item:
                     resim = item['image']

                urunler_listesi.append({
                    'Marka': item.get('brand', {}).get('name') if isinstance(item.get('brand'), dict) else item.get('brandName', 'Belirsiz'),
                    'Ürün Adı': item.get('name', 'İsimsiz'),
                    'Fiyat': fiyat,
                    'Stok': 'Var' if item.get('stockLevelStatus') == 'inStock' else 'Yok',
                    'Link': "https://www.watsons.com.tr" + item.get('url', ''),
                    'Resim': resim,
                    'Kaynak': 'JSON'
                })
    except Exception as e:
        print(f"JSON işlenirken hata oluştu: {e}")
else:
    print("❌ JSON verisi (__NEXT_DATA__) bulunamadı.")


# ---------------------------------------------------------
# YÖNTEM B: HTML ETİKETLERİNİ TARAMA (Yedek Yöntem)
# ---------------------------------------------------------
# Eğer JSON çalışmazsa, doğrudan HTML div'lerini okuruz.
if not urunler_listesi:
    print("\nYöntem B (HTML Parsing) deneniyor...")

    # Watsons ürün kartı sınıfları (Değişkenlik gösterebilir, en yaygınları ekledim)
    product_cards = soup.select('div.product-card, div.e-product-list__item, div[data-test="product-card"]')

    print(f"HTML Tarama Sonucu: {len(product_cards)} kart bulundu.")

    for card in product_cards:
        try:
            # İsim
            name_tag = card.select_one('.product-name, .product-card__name, h3')
            name = name_tag.get_text(strip=True) if name_tag else "İsimsiz"

            # Marka
            brand_tag = card.select_one('.product-brand, .product-card__brand')
            brand = brand_tag.get_text(strip=True) if brand_tag else name.split()[0]

            # Fiyat
            price_tag = card.select_one('.product-price, .price, .product-card__price')
            price = price_tag.get_text(strip=True) if price_tag else "0"

            # Link
            link_tag = card.find('a')
            link = "https://www.watsons.com.tr" + link_tag['href'] if link_tag and 'href' in link_tag.attrs else ""

            # Resim
            img_tag = card.find('img')
            img = img_tag.get('src') or img_tag.get('data-src') if img_tag else "Yok"

            urunler_listesi.append({
                'Marka': brand,
                'Ürün Adı': name,
                'Fiyat': price,
                'Stok': 'Belirsiz',
                'Link': link,
                'Resim': img,
                'Kaynak': 'HTML'
            })
        except:
            continue

# ---------------------------------------------------------
# SONUÇ VE KAYDETME
# ---------------------------------------------------------
if urunler_listesi:
    print(f"\n🎉 TOPLAM {len(urunler_listesi)} ÜRÜN AYIKLANDI.")

    df = pd.DataFrame(urunler_listesi)

    # Veriyi temizle (Kopyaları kaldır)
    df = df.drop_duplicates(subset=['Ürün Adı'])

    excel_adi = "watsons_dosyadan_cekilen.xlsx"
    df.to_excel(excel_adi, index=False)

    print(f"Dosya oluşturuldu: {excel_adi}")
    print(df.head())

    # Colab indirme kodu (Eğer localde çalıştırıyorsanız burayı silebilirsiniz)
    try:
        from google.colab import files
        files.download(excel_adi)
    except:
        pass
else:
    print("\n⚠️ Üzgünüm, bu dosyadan ürün çıkarılamadı.")
    print("Olası sebepler:")
    print("1. Dosya bir ürün listeleme sayfası değil (Anasayfa veya hata sayfası olabilir).")
    print("2. Watsons HTML yapısını değiştirmiş olabilir.")
    print("3. Dosya içeriği eksik kaydedilmiş olabilir.")

In [ ]:
# Gerekli tarayıcı ve sürücüleri yüklüyoruz
!apt-get update
!apt-get install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# --- 1. Tarayıcı Ayarları (Colab'a Özel) ---
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Tarayıcı arayüzü olmadan çalışır
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

# Sürücüyü başlat
driver = webdriver.Chrome(options=options)

# --- 2. Siteye Gitme ---
url = "https://www.watsons.com.tr/makyaj/c/100" # Çekmek istediğiniz kategori linki
print(f"Siteye bağlanılıyor: {url}")
driver.get(url)

# Sayfanın yüklenmesi için biraz bekle (Dinamik içerik)
time.sleep(5)
# Daha fazla ürün yüklenmesi için sayfayı aşağı kaydır (Scroll)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

# --- 3. Verileri Toplama ---
urunler = []

try:
    # Ürün kartlarını buluyoruz (Sınıf isimleri Watsons'ın güncel yapısına göre tahmin edilmiştir)
    # Genellikle 'product-card', 'e2e-product-card' vb. kullanılır.
    # Watsons TR genelde e2e-product-card sınıfını kullanıyor olabilir.

    # Not: Sınıf ismini bulamazsa genel bir kapsayıcı arayacağız
    kartlar = driver.find_elements(By.CSS_SELECTOR, "div[class*='product-card'], div[class*='ProductCard']")

    print(f"Toplam {len(kartlar)} ürün kartı bulundu.")

    for kart in kartlar:
        try:
            # Ürün Adı
            ad = kart.text.split('\n')[0] # Basit metin alma (Detaylı selector başarısız olursa)

            # Daha hassas veri çekimi denemesi:
            try:
                ad_element = kart.find_element(By.CSS_SELECTOR, "div[class*='name'], h3")
                ad = ad_element.text
            except:
                pass

            # Fiyat
            try:
                fiyat_element = kart.find_element(By.CSS_SELECTOR, "div[class*='price'], span[class*='sale-price']")
                fiyat = fiyat_element.text
            except:
                fiyat = "Fiyat Bulunamadı"

            urunler.append({
                "Urun_Adi": ad,
                "Fiyat": fiyat
            })

        except Exception as e:
            continue

except Exception as e:
    print(f"Hata oluştu: {e}")

finally:
    driver.quit() # Tarayıcıyı kapat

# --- 4. Sonuçları Gösterme ---
df = pd.DataFrame(urunler)

if not df.empty:
    print("\n--- Çekilen İlk 5 Ürün ---")
    print(df.head())

    # Kaydetmek isterseniz:
    # df.to_csv("watsons_canli_veri.csv", index=False)
    # print("Dosya kaydedildi.")
else:
    print("Veri çekilemedi. Sayfa yapısı değişmiş olabilir veya bot koruması engellemiş olabilir.")

In [ ]:
# Gerekli kütüphaneleri yükleyelim (Colab'da genelde yüklüdür ama garanti olsun)
!pip install beautifulsoup4 requests pandas

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

def watsons_urun_cek(url=None, html_content=None):
    """
    Watsons sitesinden ürün bilgilerini çeker.
    url: Canlı siteden çekmek için link (Örn: 'https://www.watsons.com.tr/makyaj/c/100')
    html_content: Elinizdeki HTML metnini doğrudan işlemek için (Dosya içeriği gibi)
    """

    # Eğer HTML içeriği verilmemişse, URL'den çekmeyi dene
    if html_content is None and url:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7'
        }
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                html_content = response.text
            else:
                print(f"Hata: Sayfa yüklenemedi. Durum kodu: {response.status_code}")
                return None
        except Exception as e:
            print(f"Bağlantı hatası: {e}")
            return None

    if not html_content:
        return None

    soup = BeautifulSoup(html_content, 'html.parser')
    products_data = []

    # --- YÖNTEM 1: JSON-LD Verisinden Çekme (En Temiz Yöntem) ---
    # Watsons, sayfa kaynağında SEO için hazır bir JSON verisi bulundurur.
    print("Yöntem 1: JSON-LD verisi taranıyor...")
    scripts = soup.find_all('script', {'type': 'application/ld+json'})

    for script in scripts:
        try:
            data = json.loads(script.string)
            # Ürün listesi yapısını kontrol et
            if "@type" in data and data["@type"] == "ItemList" and "itemListElement" in data:
                for item in data["itemListElement"]:
                    product = item.get("item", {})
                    if product.get("@type") == "Product":
                        p_info = {
                            "Kaynak": "JSON-LD",
                            "Ürün Adı": product.get("name"),
                            "Fiyat": product.get("offers", {}).get("price"),
                            "Para Birimi": product.get("offers", {}).get("priceCurrency"),
                            "Link": product.get("url"),
                            "Resim": product.get("image")
                        }
                        products_data.append(p_info)
        except:
            continue

    # Eğer JSON'dan veri gelmezse veya eksikse HTML elementlerini tara
    if not products_data:
        print("JSON verisi bulunamadı, Yöntem 2: HTML elementleri taranıyor...")

        # --- YÖNTEM 2: HTML Sınıflarından (Class) Çekme ---
        # Dosya incelemesine göre belirlenen sınıflar:
        # Konteyner: e2-product-tile veya .product-list-item
        product_cards = soup.select('e2-product-tile')

        for card in product_cards:
            try:
                # Ürün Adı
                name_tag = card.select_one('.product-list-item__name')
                name = name_tag.get_text(strip=True) if name_tag else "İsim Bulunamadı"

                # Link
                link_tag = card.select_one('a.product-list-item__link')
                link = "https://www.watsons.com.tr" + link_tag['href'] if link_tag else None

                # Fiyat (Normal ve İndirimli ayrımı yapılabilir)
                # Genellikle .price__default-value veya .member-price .value altında olur
                price_tag = card.select_one('.price__default-value')
                member_price_tag = card.select_one('.member-price .value')

                price = price_tag.get_text(strip=True) if price_tag else (member_price_tag.get_text(strip=True) if member_price_tag else "Fiyat Yok")

                # Resim
                img_tag = card.select_one('.product-list-item__image img')
                img_link = img_tag['src'] if img_tag and 'src' in img_tag.attrs else None

                products_data.append({
                    "Kaynak": "HTML-Parse",
                    "Ürün Adı": name,
                    "Fiyat": price,
                    "Para Birimi": "TRY",
                    "Link": link,
                    "Resim": img_link
                })
            except Exception as e:
                continue

    return pd.DataFrame(products_data)

# --- KULLANIM ---

# ÖRNEK 1: Eğer elinizde dosya içeriği bir değişkendeyse (watson.txt içeriğini buraya yapıştırabilirsiniz)
# ornek_html = """ ... watson.txt içeriği ... """
# df = watsons_urun_cek(html_content=ornek_html)

# ÖRNEK 2: Canlı siteden çekmek için (Makyaj Kategorisi)
target_url = "https://www.watsons.com.tr/makyaj/c/100"
print(f"{target_url} adresine istek atılıyor...")
df = watsons_urun_cek(url=target_url)

# Sonuçları Göster
if df is not None and not df.empty:
    print(f"\nToplam {len(df)} ürün bulundu.\n")
    # Fiyat sütununu temizle ve sayıya çevir (Analiz için)
    # Not: Canlı veride '699,90 TL' gibi stringler gelebilir, temizlemek gerekir.
    display(df.head()) # Colab'da tabloyu güzel gösterir

    # Excel'e kaydetmek isterseniz:
    # df.to_excel("watsons_urunleri.xlsx", index=False)
    # print("Excel dosyası kaydedildi.")
else:
    print("Ürün bulunamadı. Site bot koruması kullanıyor olabilir veya HTML yapısı değişmiş olabilir.")

In [ ]:
# Sistem güncellemeleri ve Chrome sürücüsünün kurulumu
!apt-get update
!apt-get install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Python kütüphanelerinin kurulumu
!pip install selenium webdriver_manager

In [ ]:
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libnss3 is already the newest version (2:3.98-0ubuntu0.22.04.2).
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.
Requirement already satisfied: selenium in /usr/local/lib/python3.12/dist-packages (4.39.0)
Requirement already satisfied: webdriver-manager in /usr/local/lib/python3.12/dist-packages (4.0.2)
Requirement already satisfied: urllib3<3.0,>=2.5.0 in /usr/local/lib/python3.12/dist-packages (from urllib3[socks]<3.0,>=2.5.0->selenium) (2.5.0)
Requirement already satisfied: trio<1.0,>=0.31.0 in /usr/local/lib/python3.12/dist-packages (from selenium) (0.32.0)
Requirement already satisfied: trio-websocket<1.0,>=0.12.2 in /usr/local/lib/python3.12/dist-packages (from selenium) (0.12.2)
Requirement already satisfied: certifi>=2025.10.5 in /usr/local/lib/python3.12/dist-packages (from selenium) (2025.11.12)
Requirement already satisfied: typing_extensions<5.0,>=4.15.0 in /usr/local/lib/python3.12/dist-packages (from selenium) (4.15.0)
Requirement already satisfied: websocket-client<2.0,>=1.8.0 in /usr/local/lib/python3.12/dist-packages (from selenium) (1.9.0)
Requirement already satisfied: requests in /usr/local/lib/python3.12/dist-packages (from webdriver-manager) (2.32.4)
Requirement already satisfied: python-dotenv in /usr/local/lib/python3.12/dist-packages (from webdriver-manager) (1.2.1)
Requirement already satisfied: packaging in /usr/local/lib/python3.12/dist-packages (from webdriver-manager) (25.0)
Requirement already satisfied: attrs>=23.2.0 in /usr/local/lib/python3.12/dist-packages (from trio<1.0,>=0.31.0->selenium) (25.4.0)
Requirement already satisfied: sortedcontainers in /usr/local/lib/python3.12/dist-packages (from trio<1.0,>=0.31.0->selenium) (2.4.0)
Requirement already satisfied: idna in /usr/local/lib/python3.12/dist-packages (from trio<1.0,>=0.31.0->selenium) (3.11)
Requirement already satisfied: outcome in /usr/local/lib/python3.12/dist-packages (from trio<1.0,>=0.31.0->selenium) (1.3.0.post0)
Requirement already satisfied: sniffio>=1.3.0 in /usr/local/lib/python3.12/dist-packages (from trio<1.0,>=0.31.0->selenium) (1.3.1)
Requirement already satisfied: wsproto>=0.14 in /usr/local/lib/python3.12/dist-packages (from trio-websocket<1.0,>=0.12.2->selenium) (1.3.2)
Requirement already satisfied: pysocks!=1.5.7,<2.0,>=1.5.6 in /usr/local/lib/python3.12/dist-packages (from urllib3[socks]<3.0,>=2.5.0->selenium) (1.7.1)
Requirement already satisfied: charset_normalizer<4,>=2 in /usr/local/lib/python3.12/dist-packages (from requests->webdriver-manager) (3.4.4)
Requirement already satisfied: h11<1,>=0.16.0 in /usr/local/lib/python3.12/dist-packages (from wsproto>=0.14->trio-websocket<1.0,>=0.12.2->selenium) (0.16.0)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# --- 1. Tarayıcıyı Hazırlama (Anti-Blok Ayarları) ---
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Arka planda çalıştır (Pencere açılmaz)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Bot gibi görünmemek için gerçek bir tarayıcı imzası kullanıyoruz
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("Tarayıcı başlatılıyor...")
driver = webdriver.Chrome(options=options)

# --- 2. Siteye Bağlanma ---
url = "https://www.watsons.com.tr/makyaj/c/100"
print(f"Siteye gidiliyor: {url}")

try:
    driver.set_page_load_timeout(30) # 30 saniye zaman aşımı sınırı
    driver.get(url)

    # Sayfanın yüklendiğini anlamak için ürün kartlarını bekle
    print("Sayfa yüklendi, ürünler aranıyor...")
    wait = WebDriverWait(driver, 20)
    # Tespit ettiğimiz doğru sınıf: 'product-list-item__info'
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-list-item__info")))

    # Daha fazla ürün yüklenmesi için sayfayı biraz aşağı kaydır
    driver.execute_script("window.scrollTo(0, 1000);")
    time.sleep(3)

    # --- 3. Ürünleri Toplama ---
    # Dosyanızdan tespit edilen güncel yapıya göre arama yapıyoruz
    urun_kartlari = driver.find_elements(By.CLASS_NAME, "product-list-item__info")
    print(f"Toplam {len(urun_kartlari)} ürün bulundu. Veriler ayıklanıyor...")

    data = []
    for kart in urun_kartlari:
        try:
            # Ürün Adı
            ad_element = kart.find_element(By.CLASS_NAME, "product-list-item__name")
            ad = ad_element.text.strip()

            # Fiyat (İndirimli veya Normal)
            # Fiyat yapısı karmaşık olduğu için genelde tüm fiyat metnini alıp temizlemek daha güvenlidir
            try:
                # Önce güncel fiyatı (member price veya sale price) bulmaya çalış
                fiyat_element = kart.find_element(By.CSS_SELECTOR, ".price__current, .price__member-price-value")
                fiyat = fiyat_element.text.strip()
            except:
                # Bulamazsa herhangi bir fiyat etiketi ara
                try:
                    fiyat_element = kart.find_element(By.CLASS_NAME, "product-info__price")
                    fiyat = fiyat_element.text.strip()
                except:
                    fiyat = "Fiyat Görünmedi"

            data.append({"Urun_Adi": ad, "Fiyat": fiyat})

        except Exception as inner_e:
            continue # Bozuk bir kart varsa geç

    # --- 4. Sonuçları Gösterme ---
    df = pd.DataFrame(data)
    print("\n--- Çekilen İlk 5 Ürün ---")
    print(df.head())

    # İsterseniz CSV olarak kaydedin
    df.to_csv("watsons_canli_urunler.csv", index=False)
    print("\nVeriler 'watsons_canli_urunler.csv' dosyasına kaydedildi.")

except Exception as e:
    print(f"\nHATA OLUŞTU: {e}")
    print("İpucu: Eğer 'Timeout' hatası alıyorsanız, Colab IP'si geçici olarak engellenmiş olabilir.")

finally:
    driver.quit()
    print("Tarayıcı kapatıldı.")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# --- 1. Tarayıcı Ayarları ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=options)

# --- 2. Siteye Gitme ---
url = "https://www.watsons.com.tr/makyaj/c/100"
print(f"Siteye bağlanılıyor: {url}")
driver.get(url)

# Sayfanın ilk yüklenişini bekle
wait = WebDriverWait(driver, 20)
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-list-item__info")))
time.sleep(3)

# --- 3. SONSUZ KAYDIRMA DÖNGÜSÜ (Tüm Ürünleri Yüklemek İçin) ---
print("Sayfa aşağı kaydırılıyor (Tüm ürünler yükleniyor)...")

last_height = driver.execute_script("return document.body.scrollHeight")
scroll_attempt = 0

while True:
    # Sayfanın en altına in
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Yüklenme için bekle (İnternet hızına göre artırılabilir)
    time.sleep(3)

    # Yeni sayfa yüksekliğini al
    new_height = driver.execute_script("return document.body.scrollHeight")

    # Eğer yükseklik değişmediyse (sayfa sonuna geldik demektir)
    if new_height == last_height:
        # Bazen 'Daha Fazla' butonu olabilir veya yükleme takılabilir, emin olmak için 1-2 kere daha dene
        scroll_attempt += 1
        if scroll_attempt >= 3: # 3 kere denemesine rağmen yükseklik artmıyorsa bitir
            break
    else:
        scroll_attempt = 0 # Yükleme başarılıysa sayacı sıfırla
        last_height = new_height
        # İlerleme durumunu görmek için mevcut ürün sayısını yazdırabiliriz
        urun_sayisi = len(driver.find_elements(By.CLASS_NAME, "product-list-item__info"))
        print(f"Kaydırılıyor... Şu ana kadar yüklenen tahmini ürün: {urun_sayisi}")

print("Kaydırma tamamlandı. Veriler toplanıyor...")

# --- 4. Verileri Toplama ---
urun_kartlari = driver.find_elements(By.CLASS_NAME, "product-list-item__info")
print(f"Toplam {len(urun_kartlari)} ürün bulundu.")

data = []
for kart in urun_kartlari:
    try:
        # Ürün Adı
        ad = kart.find_element(By.CLASS_NAME, "product-list-item__name").text.strip()

        # Fiyat
        try:
            # Önce indirimli/güncel fiyatı dene
            fiyat = kart.find_element(By.CSS_SELECTOR, ".price__current, .price__member-price-value").text.strip()
        except:
            try:
                # Bulamazsa genel fiyat etiketini dene
                fiyat = kart.find_element(By.CLASS_NAME, "product-info__price").text.strip()
            except:
                fiyat = "Fiyat Yok"

        data.append({"Urun_Adi": ad, "Fiyat": fiyat})

    except:
        continue

driver.quit()

# --- 5. Kaydetme ve Gösterme ---
df = pd.DataFrame(data)

# Fiyat temizliği (Örn: '1.229,90 ₺' -> 1229.90 float formatına çevirme)
# Analiz yapacaksanız bu adım gereklidir.
def fiyat_temizle(fiyat_str):
    if not isinstance(fiyat_str, str): return 0
    clean = fiyat_str.replace('₺', '').replace('.', '').replace(',', '.').strip()
    try:
        return float(clean)
    except:
        return 0

# Temiz fiyat sütunu ekleyelim (Opsiyonel)
df['Fiyat_Sayisal'] = df['Fiyat'].apply(fiyat_temizle)

print("\n--- Çekilen İlk 5 Ürün ---")
print(df.head())

print(f"\nToplam {len(df)} ürün kaydedildi.")
df.to_csv("watsons_tum_urunler.csv", index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# --- AYARLAR ---
# Kaç sayfa çekmek istiyorsunuz? (Tüm ürünler için yaklaşık 140 sayfa vardır: 3905/28)
# Deneme için önce 3 yapın, çalışıyorsa 150 yapıp bırakın.
CEKILECEK_SAYFA_SAYISI = 3

# --- 1. Tarayıcı Ayarları ---
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Arka planda çalışır
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Resimlerin yüklenmesi için pencere boyutunu büyük tutuyoruz
options.add_argument("window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=options)
tum_urunler = []

print(f"Toplam {CEKILECEK_SAYFA_SAYISI} sayfa taranacak...")

# --- 2. Sayfa Döngüsü ---
for sayfa_no in range(CEKILECEK_SAYFA_SAYISI):
    # Watsons sayfa yapısı genelde ?page=0, ?page=1 şeklindedir.
    url = f"https://www.watsons.com.tr/makyaj/c/100?page={sayfa_no}"
    print(f"\n--- Sayfa {sayfa_no + 1} işleniyor ({url}) ---")

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        # Ürünlerin kapsayıcısının yüklenmesini bekle
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-list-item__info")))

        # --- Resimlerin Yüklenmesi İçin Kaydırma ---
        # Sayfayı yavaşça aşağı kaydırıyoruz ki 'Lazy Load' resimler yüklensin
        for i in range(1, 6):
            driver.execute_script(f"window.scrollTo(0, {i * 800});")
            time.sleep(1) # Resmin yüklenmesi için kısa bekleme

        # En üste geri çık (Bazen elementler kaybolmasın diye)
        # driver.execute_script("window.scrollTo(0, 0);")

        # --- Verileri Toplama ---
        # Ürün kartlarını bul
        kartlar = driver.find_elements(By.CLASS_NAME, "product-list-item__info")
        print(f"Sayfada {len(kartlar)} ürün bulundu.")

        for kart in kartlar:
            try:
                # 1. İsim
                ad = kart.find_element(By.CLASS_NAME, "product-list-item__name").text.strip()

                # 2. Fiyat
                try:
                    # İndirimli/Üye fiyatı öncelikli
                    fiyat = kart.find_element(By.CSS_SELECTOR, ".price__current, .price__member-price-value").text.strip()
                except:
                    try:
                        fiyat = kart.find_element(By.CLASS_NAME, "product-info__price").text.strip()
                    except:
                        fiyat = "Fiyat Yok"

                # 3. Resim Linki (YENİ)
                resim_url = "Resim Yok"
                try:
                    # Kartın içindeki img etiketini bul
                    # Watsons'da resimler genellikle bir üst div'de olabilir, kartın tamamına bakıyoruz
                    # .product-list-item__info'nun bir üstüne (parent) çıkıp oradan resme ulaşmak daha garantidir
                    parent_card = kart.find_element(By.XPATH, "./..")
                    resim_element = parent_card.find_element(By.TAG_NAME, "img")
                    resim_url = resim_element.get_attribute("src")

                    # Eğer lazy load varsa src boş olabilir, data-src'ye bakılabilir
                    if not resim_url or "base64" in resim_url:
                        resim_url = resim_element.get_attribute("data-src")

                except Exception as img_err:
                    # Bazen resim class'ı farklı olabilir
                    pass

                tum_urunler.append({
                    "Sayfa": sayfa_no + 1,
                    "Urun_Adi": ad,
                    "Fiyat": fiyat,
                    "Resim_URL": resim_url
                })

            except:
                continue

    except Exception as e:
        print(f"Sayfa {sayfa_no + 1} yüklenirken hata: {e}")
        continue

driver.quit()

# --- 3. Kaydetme ve Gösterme ---
df = pd.DataFrame(tum_urunler)

print(f"\nİşlem Tamamlandı! Toplam {len(df)} ürün çekildi.")
print("--- Örnek Veri ---")
print(df[["Urun_Adi", "Fiyat", "Resim_URL"]].head())

# CSV'ye kaydet
df.to_csv("watsons_resimli_urunler.csv", index=False)
print("Dosya kaydedildi: watsons_resimli_urunler.csv")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# --- AYARLAR ---
# Test için 3 yaptık. Tüm ürünler (yaklaşık 140 sayfa) için bu sayıyı artırın (örn: 140).
CEKILECEK_SAYFA_SAYISI = 3

# --- 1. Tarayıcı Ayarları ---
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Tarayıcıyı ekranda göstermeden arka planda çalıştırır
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Resimlerin yüklenmesi için pencere boyutunu büyük tutuyoruz
options.add_argument("window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("Tarayıcı başlatılıyor...")
driver = webdriver.Chrome(options=options)
tum_urunler = []

print(f"Toplam {CEKILECEK_SAYFA_SAYISI} sayfa taranacak...")

# --- 2. Sayfa Döngüsü ---
for sayfa_no in range(CEKILECEK_SAYFA_SAYISI):
    # Sayfa URL'sini oluştur
    url = f"https://www.watsons.com.tr/makyaj/c/100?page={sayfa_no}"
    print(f"\n--- Sayfa {sayfa_no + 1} işleniyor... ---")

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        # Ürünlerin yüklenmesini bekle
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-list-item__info")))

        # --- Resimlerin Yüklenmesi İçin Kaydırma (Scroll) ---
        # Sayfayı parça parça aşağı kaydırıyoruz
        for i in range(1, 5):
            driver.execute_script(f"window.scrollTo(0, {i * 1000});")
            time.sleep(1.5) # Resimlerin yüklenmesi için bekleme

        # --- Verileri Toplama ---
        kartlar = driver.find_elements(By.CLASS_NAME, "product-list-item__info")
        print(f"   -> {len(kartlar)} ürün bulundu.")

        for kart in kartlar:
            try:
                # 1. Ürün Adı
                try:
                    ad = kart.find_element(By.CLASS_NAME, "product-list-item__name").text.strip()
                except:
                    ad = "İsimsiz Ürün"

                # 2. Fiyat
                try:
                    # İndirimli/Üye fiyatı öncelikli
                    fiyat = kart.find_element(By.CSS_SELECTOR, ".price__current, .price__member-price-value").text.strip()
                except:
                    try:
                        fiyat = kart.find_element(By.CLASS_NAME, "product-info__price").text.strip()
                    except:
                        fiyat = "Fiyat Yok"

                # 3. Resim Linki
                resim_url = "Resim Yok"
                try:
                    # Kartın bulunduğu ana kutuya çıkıp img etiketini arıyoruz
                    parent_card = kart.find_element(By.XPATH, "./..")
                    resim_element = parent_card.find_element(By.TAG_NAME, "img")

                    # Önce 'src' al, eğer boşsa veya yüklenmemişse 'data-src' al
                    resim_url = resim_element.get_attribute("src")
                    if not resim_url or "base64" in resim_url or "gif" in resim_url:
                        data_src = resim_element.get_attribute("data-src")
                        if data_src:
                            resim_url = data_src

                except:
                    pass

                tum_urunler.append({
                    "Sayfa": sayfa_no + 1,
                    "Urun_Adi": ad,
                    "Fiyat": fiyat,
                    "Resim_URL": resim_url
                })

            except:
                continue

    except Exception as e:
        print(f"   !!! Sayfa {sayfa_no + 1} yüklenirken hata oluştu: {e}")
        continue

driver.quit()
print("\nTarayıcı kapatıldı.")

# --- 3. Kaydetme ve Gösterme ---
if tum_urunler:
    df = pd.DataFrame(tum_urunler)
    print(f"\nİşlem Başarılı! Toplam {len(df)} ürün çekildi.")
    print("--- İlk 5 Ürün ---")
    print(df[["Urun_Adi", "Fiyat", "Resim_URL"]].head())

    # CSV olarak kaydet
    csv_adi = "watsons_resimli_urunler.csv"
    df.to_csv(csv_adi, index=False)
    print(f"\nDosya kaydedildi: {csv_adi}")

    # Dosyayı bilgisayara indirmek için (Colab)
    from google.colab import files
    files.download(csv_adi)
else:
    print("Hiçbir ürün çekilemedi. Lütfen site yapısını veya bağlantınızı kontrol edin.")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# --- AYARLAR ---
# Kaç sayfa çekecekseniz burayı güncelleyin (Tüm ürünler için ~140)
CEKILECEK_SAYFA_SAYISI = 192

# --- 1. Tarayıcı Ayarları ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("Tarayıcı başlatılıyor...")
driver = webdriver.Chrome(options=options)
tum_urunler = []

# --- 2. Sayfa Döngüsü ---
for sayfa_no in range(CEKILECEK_SAYFA_SAYISI):
    url = f"https://www.watsons.com.tr/makyaj/c/100?page={sayfa_no}"
    print(f"\n--- Sayfa {sayfa_no + 1} işleniyor... ---")

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        # En dıştaki ürün kartının yüklenmesini bekliyoruz
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "product-list-item")))

        # --- Resimlerin Yüklenmesi İçin Kaydırma ---
        for i in range(1, 5):
            driver.execute_script(f"window.scrollTo(0, {i * 1000});")
            time.sleep(1.5)

        # --- Verileri Toplama (DEĞİŞİKLİK BURADA) ---
        # Artık sadece "info" kısmını değil, tüm kartı seçiyoruz
        kartlar = driver.find_elements(By.CLASS_NAME, "product-list-item")
        print(f"   -> {len(kartlar)} ürün kartı bulundu.")

        for kart in kartlar:
            try:
                # 1. Ürün Adı
                try:
                    ad = kart.find_element(By.CLASS_NAME, "product-list-item__name").text.strip()
                except:
                    ad = "İsimsiz Ürün"

                # 2. Fiyat
                try:
                    fiyat = kart.find_element(By.CSS_SELECTOR, ".price__current, .price__member-price-value").text.strip()
                except:
                    try:
                        fiyat = kart.find_element(By.CLASS_NAME, "product-info__price").text.strip()
                    except:
                        fiyat = "Fiyat Yok"

                # 3. Resim Linki (GÜNCELLENDİ)
                resim_url = "Resim Yok"
                try:
                    # Kartın içindeki resim konteynerine odaklanıyoruz
                    # .product-list-item__image-container sınıfı resmin olduğu yerdir
                    resim_container = kart.find_element(By.CLASS_NAME, "product-list-item__image-container")
                    img_element = resim_container.find_element(By.TAG_NAME, "img")

                    # src veya data-src kontrolü
                    src = img_element.get_attribute("src")
                    data_src = img_element.get_attribute("data-src")

                    # Eğer src varsa ve geçerliyse (base64 değilse) al, yoksa data-src al
                    if src and "http" in src:
                        resim_url = src
                    elif data_src and "http" in data_src:
                        resim_url = data_src
                    elif src: # Mecburen ne varsa al
                         resim_url = src

                except Exception as img_err:
                    pass

                tum_urunler.append({
                    "Sayfa": sayfa_no + 1,
                    "Urun_Adi": ad,
                    "Fiyat": fiyat,
                    "Resim_URL": resim_url
                })

            except:
                continue

    except Exception as e:
        print(f"   !!! Sayfa {sayfa_no + 1} hatası: {e}")
        continue

driver.quit()
print("\nTarayıcı kapatıldı.")

# --- 3. Sonuçları Kaydetme ---
if tum_urunler:
    df = pd.DataFrame(tum_urunler)
    print(f"\nToplam {len(df)} ürün çekildi.")
    print("--- Örnek Veri (İlk 3) ---")
    print(df[["Urun_Adi", "Resim_URL"]].head(3))

    df.to_csv("watsons_resimli_urunler_v2.csv", index=False)
    print("\nDosya kaydedildi: watsons_resimli_urunler_v2.csv")

    from google.colab import files
    files.download("watsons_resimli_urunler_v2.csv")
else:
    print("Ürün bulunamadı.")

In [ ]:
# --- 2. MASTER VERİ ÇEKME KODU (DÜZELTİLMİŞ) ---
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os

# --- LİSTENECEK LİNKLER ---
kategori_linkleri = [
    "https://www.watsons.com.tr/makyaj/c/100",
    "https://www.watsons.com.tr/cilt-bakim/c/101",
    "https://www.watsons.com.tr/sac-bakim/c/102",
    "https://www.watsons.com.tr/kisisel-bakim/c/103",
    "https://www.watsons.com.tr/saglikli-yasam/c/106",
    "https://www.watsons.com.tr/erkek-bakim/c/105",
    "https://www.watsons.com.tr/parfum/c/100013",
    "https://www.watsons.com.tr/aksesuar/c/109",
    "https://www.watsons.com.tr/k-beauty/c/104",
    "https://www.watsons.com.tr/surdurulebilir-yasam"
]

# --- TARAYICI AYARLARI ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("🚀 Tarayıcı başlatılıyor...")
driver = webdriver.Chrome(options=options)

tum_veriler = []
dosya_adi = "watsons_tum_kategoriler.csv"

# --- ANA DÖNGÜ (HER KATEGORİ İÇİN) ---
for ana_link in kategori_linkleri:
    try:
        kategori_adi = ana_link.split(".tr/")[1].split("/")[0]
    except:
        kategori_adi = "Genel"

    print(f"\n📂 KATEGORİ BAŞLIYOR: {kategori_adi.upper()} ({ana_link})")

    sayfa_no = 0
    onceki_sayfa_ilk_urun = "" # Döngü kontrolü için değişken

    while True:
        url = f"{ana_link}?page={sayfa_no}"

        try:
            driver.get(url)
            time.sleep(2)

            # Kartları bul
            kartlar = driver.find_elements(By.CLASS_NAME, "product-list-item")

            # KONTROL 1: Kart sayısı 0 ise bitir
            if len(kartlar) == 0:
                print(f"   🛑 {kategori_adi} bitti (Kart yok). Toplam sayfa: {sayfa_no}")
                break

            # KONTROL 2 (YENİ): Bu sayfanın ilk ürününün adı, bir önceki sayfayla aynı mı?
            # Watsons sayfa bitince başa dönüyorsa bunu yakalarız.
            try:
                suanki_ilk_urun = kartlar[0].find_element(By.CLASS_NAME, "product-list-item__name").text.strip()
            except:
                suanki_ilk_urun = "Bilinmiyor"

            if suanki_ilk_urun == onceki_sayfa_ilk_urun and sayfa_no > 0:
                print(f"   🛑 {kategori_adi} bitti (İçerik tekrar etti). Toplam sayfa: {sayfa_no}")
                break

            # Mevcut ilk ürünü hafızaya al
            onceki_sayfa_ilk_urun = suanki_ilk_urun

            print(f"   📄 Sayfa {sayfa_no} taranıyor... ({len(kartlar)} ürün) - İlk Ürün: {suanki_ilk_urun[:15]}...")

            # --- Lazy Load İçin Kaydırma ---
            for i in range(1, 4):
                driver.execute_script(f"window.scrollTo(0, {i * 1200});")
                time.sleep(0.5) # Süreyi biraz kısalttım hızlansın diye

            # --- Verileri Çekme ---
            sayfa_verisi = []
            for kart in kartlar:
                try:
                    try:
                        ad = kart.find_element(By.CLASS_NAME, "product-list-item__name").text.strip()
                    except:
                        ad = "İsimsiz"

                    try:
                        # Fiyat class'ları bazen değişebilir, çoklu kontrol
                        fiyat = kart.find_element(By.CSS_SELECTOR, ".price__current, .price__member-price-value, .product-info__price").text.strip()
                    except:
                        fiyat = "Tükendi/Fiyat Yok"

                    resim_url = "Resim Yok"
                    try:
                        resim_container = kart.find_element(By.CLASS_NAME, "product-list-item__image-container")
                        img = resim_container.find_element(By.TAG_NAME, "img")
                        src = img.get_attribute("src")
                        data_src = img.get_attribute("data-src")

                        if src and "http" in src: resim_url = src
                        elif data_src and "http" in data_src: resim_url = data_src
                    except:
                        pass

                    sayfa_verisi.append({
                        "Kategori": kategori_adi,
                        "Sayfa": sayfa_no,
                        "Urun_Adi": ad,
                        "Fiyat": fiyat,
                        "Resim_URL": resim_url,
                        "Urun_Link": url
                    })
                except:
                    continue

            tum_veriler.extend(sayfa_verisi)
            sayfa_no += 1

        except Exception as e:
            print(f"   ⚠️ Hata: {e}. Sonraki kategoriye geçiliyor.")
            break

    # Her kategori bitişinde kaydet
    df_temp = pd.DataFrame(tum_veriler)
    df_temp.to_csv(dosya_adi, index=False)
    print(f"💾 '{kategori_adi}' kaydedildi. Toplam satır: {len(df_temp)}")

driver.quit()
print(f"\n✅ İŞLEM BİTTİ! Toplam {len(tum_veriler)} veri çekildi.")

# İndir
from google.colab import files
files.download(dosya_adi)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import math
import re

# --- LİSTENECEK LİNKLER ---
kategori_linkleri = [
    "https://www.watsons.com.tr/makyaj/c/100",
    "https://www.watsons.com.tr/cilt-bakim/c/101",
    "https://www.watsons.com.tr/sac-bakim/c/102",
    "https://www.watsons.com.tr/kisisel-bakim/c/103",
    "https://www.watsons.com.tr/saglikli-yasam/c/106",
    "https://www.watsons.com.tr/erkek-bakim/c/105",
    "https://www.watsons.com.tr/parfum/c/100013",
    "https://www.watsons.com.tr/aksesuar/c/109",
    "https://www.watsons.com.tr/k-beauty/c/104",
    "https://www.watsons.com.tr/surdurulebilir-yasam"
]

# --- TARAYICI AYARLARI ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("🚀 Tarayıcı başlatılıyor...")
driver = webdriver.Chrome(options=options)

tum_veriler = []
dosya_adi = "watsons_net_sayili.csv"

# --- ANA DÖNGÜ ---
for ana_link in kategori_linkleri:
    try:
        kategori_adi = ana_link.split(".tr/")[1].split("/")[0]
    except:
        kategori_adi = "Genel"

    print(f"\n📂 KATEGORİ BAŞLIYOR: {kategori_adi.upper()}")

    # -------------------------------------------------------------
    # ADIM 1: KATEGORİDEKİ TOPLAM SAYIYI BULMA
    # -------------------------------------------------------------
    driver.get(f"{ana_link}?page=0")
    time.sleep(4) # Sayfanın ve sayacın yüklenmesi için bekle

    toplam_urun_sayisi = 0
    hedeflenen_sayfa_sayisi = 1 # Varsayılan en az 1

    try:
        # Sayfanın tüm metin içeriğini al
        sayfa_metni = driver.find_element(By.TAG_NAME, "body").text

        # Regex ile "1048 Ürün" veya "1048 results" yapısını ara
        # Watsons genelde "1234 Ürün" yazar. Nokta varsa kaldırırız (1.234 -> 1234)
        match = re.search(r'(\d+[.,]?\d*)\s+Ürün', sayfa_metni, re.IGNORECASE)

        if match:
            # Sayıdaki noktaları temizle (1.048 -> 1048)
            temiz_sayi = match.group(1).replace('.', '').replace(',', '')
            toplam_urun_sayisi = int(temiz_sayi)

            # Watsons her sayfada 24 ürün gösterir
            hedeflenen_sayfa_sayisi = math.ceil(toplam_urun_sayisi / 24)

            print(f"   📊 TESPİT EDİLDİ: Toplam {toplam_urun_sayisi} ürün var.")
            print(f"   🎯 HEDEF: {hedeflenen_sayfa_sayisi} sayfa gezilecek.")
        else:
            print("   ⚠️ Sayı etiketi bulunamadı, manuel modda ilerlenecek.")
            hedeflenen_sayfa_sayisi = 500 # Bulamazsa güvenlik sınırı

    except Exception as e:
        print(f"   ⚠️ Sayı okuma hatası: {e}")

    # -------------------------------------------------------------
    # ADIM 2: HESAPLANAN SAYFA SAYISI KADAR DÖNGÜ
    # -------------------------------------------------------------

    toplanan_urun_sayisi = 0

    # Range fonksiyonu 0'dan başlar, hedef sayıya kadar gider.
    # Örn: 44 sayfa ise 0'dan 43'e kadar (URL page=0 ... page=43)
    for sayfa_no in range(hedeflenen_sayfa_sayisi):

        # Eğer manuel moddaysak ve ürün sayısı hedefe ulaştıysa dur
        if toplanan_urun_sayisi >= toplam_urun_sayisi and toplam_urun_sayisi > 0:
            print(f"   ✅ Hedeflenen ürün sayısına ({toplam_urun_sayisi}) ulaşıldı.")
            break

        url = f"{ana_link}?page={sayfa_no}"

        try:
            driver.get(url)
            # Sayfa geçişlerinde kısa bekleme
            time.sleep(1.5)

            # Kartları bul
            kartlar = driver.find_elements(By.CLASS_NAME, "product-list-item")

            # Boş sayfa kontrolü (Sayıyı yanlış hesapladıysak durması için)
            if len(kartlar) == 0:
                print(f"   🛑 Sayfa {sayfa_no} boş geldi. Kategori erken bitti.")
                break

            print(f"   📄 Sayfa {sayfa_no+1}/{hedeflenen_sayfa_sayisi} taranıyor... ({len(kartlar)} ürün eklendi)")

            # --- Lazy Load Kaydırma ---
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.8)

            # --- Verileri Topla ---
            sayfa_verisi = []
            for kart in kartlar:
                try:
                    ad = kart.find_element(By.CLASS_NAME, "product-list-item__name").text.strip()

                    try:
                        fiyat = kart.find_element(By.CSS_SELECTOR, ".price__current, .price__member-price-value, .product-info__price").text.strip()
                    except:
                        fiyat = "Fiyat Yok"

                    resim_url = "Resim Yok"
                    try:
                        img_element = kart.find_element(By.CSS_SELECTOR, ".product-list-item__image-container img")
                        src = img_element.get_attribute("src")
                        if src: resim_url = src
                    except:
                        pass

                    # Link
                    try:
                        link_tag = kart.find_element(By.TAG_NAME, "a")
                        urun_link = link_tag.get_attribute("href")
                    except:
                        urun_link = url

                    sayfa_verisi.append({
                        "Kategori": kategori_adi,
                        "Urun_Adi": ad,
                        "Fiyat": fiyat,
                        "Resim_URL": resim_url,
                        "Urun_Link": urun_link
                    })
                except:
                    continue

            tum_veriler.extend(sayfa_verisi)
            toplanan_urun_sayisi += len(sayfa_verisi)

        except Exception as e:
            print(f"   ⚠️ Sayfa hatası: {e}")
            continue

    # Kategori bitince kaydet
    df_temp = pd.DataFrame(tum_veriler)
    df_temp.to_csv(dosya_adi, index=False)
    print(f"💾 '{kategori_adi}' tamamlandı. Toplam {toplanan_urun_sayisi} ürün çekildi.\n")

driver.quit()
print(f"\n✅ TÜM İŞLEMLER BİTTİ! Toplam {len(tum_veriler)} satır.")

# İndir
from google.colab import files
files.download(dosya_adi)

In [ ]:
import pandas as pd
import sqlite3
import re

# 1. CSV DOSYASINI OKU
dosya_adi = "watsons_net_sayili.csv"
try:
    df = pd.read_csv(dosya_adi)
    print(f"✅ CSV okundu. Toplam satır: {len(df)}")
except FileNotFoundError:
    print("❌ Hata: CSV dosyası bulunamadı. Lütfen 'watsons_net_sayili.csv' dosyasını yükleyin.")

# ---------------------------------------------------------
# 2. VERİ TEMİZLEME (TRANSFORMATION)
# ---------------------------------------------------------
# SQL'de analiz yapabilmek için fiyatın "Yazı" değil "Sayı" olması lazım.
# Örnek Veri: "1.299,90 ₺\nWatsons Club ile\n649,90 ₺" -> Hedef: 649.90

def fiyat_temizle(fiyat_metni):
    if pd.isna(fiyat_metni):
        return None

    # Tüm sayıları bul (Örn: ['1.299,90', '649,90'])
    # Nokta ve virgül karmaşasını regex ile çözelim
    bulunanlar = re.findall(r'(\d+[.,\d]*)', str(fiyat_metni))

    if not bulunanlar:
        return None

    temiz_fiyatlar = []
    for f in bulunanlar:
        # Binlik ayracı (.) kaldır, Kuruş ayracı (,) nokta yap
        try:
            temiz = float(f.replace('.', '').replace(',', '.'))
            temiz_fiyatlar.append(temiz)
        except:
            continue

    # Eğer birden fazla fiyat varsa (İndirimli/İndirimsiz), en düşüğünü al
    if temiz_fiyatlar:
        return min(temiz_fiyatlar)
    return None

# Fonksiyonu uygula
print("🧹 Veri temizleniyor (ETL Süreci)...")
df['Satis_Fiyati'] = df['Fiyat'].apply(fiyat_temizle)

# Kategori ismini temizle (küçük harf, boşlukları düzelt)
df['Kategori'] = df['Kategori'].str.lower().str.strip()

# İşe yaramayan (Fiyatı çekilememiş) satırları at
df_clean = df.dropna(subset=['Satis_Fiyati'])

print(f"✅ Temizlik bitti. {len(df) - len(df_clean)} hatalı satır çıkarıldı. Kalan: {len(df_clean)}")

# ---------------------------------------------------------
# 3. VERİTABANINA YÜKLEME (LOAD)
# ---------------------------------------------------------
db_adi = "rossmann_pazar_analizi.db"
conn = sqlite3.connect(db_adi)
cursor = conn.cursor()

# Tabloyu oluştur (Eğer varsa önce silip yeniden kurar)
cursor.execute("DROP TABLE IF EXISTS urunler")

# Pandas'ın sihirli fonksiyonu ile veriyi SQL'e dök
# index=False -> Pandas satır numaralarını kaydetme
df_clean.to_sql('urunler', conn, if_exists='replace', index=False)

print(f"💾 Veriler '{db_adi}' veritabanındaki 'urunler' tablosuna aktarıldı.")

# ---------------------------------------------------------
# 4. TEST SORGUSU (Mülakatta Göstereceğin Kısım)
# ---------------------------------------------------------
print("\n🔍 SQL TESTİ: Kategori Bazlı Ortalama Fiyatlar")
query = """
SELECT
    Kategori,
    COUNT(*) as Urun_Adedi,
    ROUND(AVG(Satis_Fiyati), 2) as Ortalama_Fiyat,
    MIN(Satis_Fiyati) as En_Ucuz,
    MAX(Satis_Fiyati) as En_Pahali
FROM urunler
GROUP BY Kategori
ORDER BY Ortalama_Fiyat DESC
"""

sql_sonuc = pd.read_sql(query, conn)
display(sql_sonuc)

conn.close()

# İndirme Butonu
from google.colab import files
files.download(db_adi)

In [ ]:
import pandas as pd
import re
import datetime
from IPython.display import display

# --- 1. VERİ OKUMA (Extract) ---
dosya_adi = "watsons_net_sayayli.csv"
try:
    df = pd.read_csv(dosya_adi)
    print(f"✅ CSV okundu. Başlangıç satır sayısı: {len(df)}")
except FileNotFoundError:
    print("❌ Hata: CSV dosyası bulunamadı. Lütfen 'watsons_net_sayili.csv' dosyasının yüklü olduğundan emin olun.")
    exit()

# --- 2. VERİ DÖNÜŞÜMÜ (Transform) ---

print("🧹 ETL: Fiyat ve Marka kolonları ayrıştırılıyor...")

# A) Fiyat Ayırma Fonksiyonu (Normal ve İndirimli)
def fiyat_ayirici(fiyat_metni):
    if pd.isna(fiyat_metni): return None, None

    # Regex ile tüm sayısal fiyatları bul
    bulunanlar = re.findall(r'(\d{1,3}(?:\.\d{3})*,\d{2})', str(fiyat_metni))
    temiz_fiyatlar = []

    for f in bulunanlar:
        try:
            temiz = float(f.replace('.', '').replace(',', '.'))
            temiz_fiyatlar.append(temiz)
        except:
            continue

    # Atama Mantığı: En büyük = Normal, En küçük = İndirimli
    if len(temiz_fiyatlar) >= 2:
        return max(temiz_fiyatlar), min(temiz_fiyatlar)
    elif len(temiz_fiyatlar) == 1:
        return temiz_fiyatlar[0], None
    else:
        return None, None

# B) Marka Ayırma Fonksiyonu
def marka_ayirici(urun_adi):
    """
    Ürün adının ilk iki kelimesini marka olarak kabul eder.
    (Örn: L'Oreal Paris -> L'Oreal Paris)
    """
    if not isinstance(urun_adi, str): return None

    words = urun_adi.strip().split()

    if len(words) >= 2:
        # İlk iki kelimeyi al (Çoğu marka 1 veya 2 kelimedir)
        return " ".join(words[:2])
    elif len(words) == 1:
        return words[0]
    return None

# C) Dönüşümleri Uygula
df[['normal_fiyat', 'indirimli_fiyat']] = df['Fiyat'].apply(
    lambda x: pd.Series(fiyat_ayirici(x))
)

df['marka'] = df['Urun_Adi'].apply(marka_ayirici) # YENİ MARKA KOLONU
df['kategori'] = df['Kategori'].str.lower().str.strip().str.replace('-', '_')
df['urun_adi'] = df['Urun_Adi'].str.strip()
df['resim_url'] = df['Resim_URL'].str.strip()
df['urun_link'] = df['Urun_Link'].str.strip()
df['urun_tarihi'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# D) HEDEF KOLON SIRALAMASI ve SON TEMİZLİK
# watsons_products tablosunun beklediği KOLON SIRA ve İSİMLERİ
df_final = df[[
    'kategori',
    'marka',            # YENİ MARKAYI EŞLEŞTİR
    'urun_adi',
    'normal_fiyat',
    'indirimli_fiyat',
    'resim_url',
    'urun_link',
    'urun_tarihi'
]].copy()

# Fiyatı olmayan veya hatalı olan satırları at
df_final = df_final.dropna(subset=['normal_fiyat'])
df_final = df_final[df_final['normal_fiyat'] > 0]

# --- 3. SONUCU GÖSTERME ---
print(f"✅ ETL Tamamlandı. Veritabanına hazır satır: {len(df_final)}\n")

print("--- SQL Tablosuna Hazır İlk 5 Satır ---")
# Marka, normal ve indirimli fiyatın ayrıldığını kontrol edelim
display(df_final.head())

In [ ]:
import pandas as pd
import re
import datetime
from IPython.display import display

# --- 1. VERİ OKUMA (Extract) ---
dosya_adi = "watsons_net_sayili.csv"
Cikis_Dosya_Adi = "watsons_analize_hazir_veri.csv"

try:
    df = pd.read_csv(dosya_adi)
    print(f"✅ CSV okundu. Başlangıç satır sayısı: {len(df)}")
except FileNotFoundError:
    print("❌ Hata: CSV dosyası bulunamadı. Lütfen kontrol edin.")
    exit()

# --- 2. VERİ DÖNÜŞÜMÜ (Transform) ---

print("🧹 ETL: Veri temizleniyor (Marka ve Fiyat ayrımı yapılıyor)...")

# A) Fiyat Ayırma Fonksiyonu (Normal ve İndirimli)
def fiyat_ayirici(fiyat_metni):
    if pd.isna(fiyat_metni): return None, None

    # Regex ile tüm sayısal fiyatları bul (X.XXX,XX formatında)
    bulunanlar = re.findall(r'(\d{1,3}(?:\.\d{3})*,\d{2})', str(fiyat_metni))
    temiz_fiyatlar = []

    for f in bulunanlar:
        try:
            # Noktayı kaldır, virgülü nokta yap (Sayısal format)
            temiz = float(f.replace('.', '').replace(',', '.'))
            temiz_fiyatlar.append(temiz)
        except:
            continue

    # Atama Mantığı: En büyük = Normal Fiyat, En küçük = İndirimli Fiyat
    if len(temiz_fiyatlar) >= 2:
        return max(temiz_fiyatlar), min(temiz_fiyatlar)
    elif len(temiz_fiyatlar) == 1:
        return temiz_fiyatlar[0], None # İndirim yoksa NULL
    else:
        return None, None

# B) Marka Ayırma Fonksiyonu
def marka_ayirici(urun_adi):
    """Ürün adının ilk iki kelimesini marka olarak kabul eder."""
    if not isinstance(urun_adi, str): return None

    words = urun_adi.strip().split()

    if len(words) >= 2:
        return " ".join(words[:2])
    elif len(words) == 1:
        return words[0]
    return None

# C) Tüm Dönüşümleri Uygula ve Yeni Kolonları Oluştur
df[['normal_fiyat', 'indirimli_fiyat']] = df['Fiyat'].apply(
    lambda x: pd.Series(fiyat_ayirici(x))
)

df['marka'] = df['Urun_Adi'].apply(marka_ayirici)
df['kategori'] = df['Kategori'].str.lower().str.strip().str.replace('-', '_')
df['urun_adi'] = df['Urun_Adi'].str.strip()
df['resim_url'] = df['Resim_URL'].str.strip()
df['urun_link'] = df['Urun_Link'].str.strip()
df['urun_tarihi'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') # Veri zaman etiketi

# D) HEDEF KOLON SIRALAMASI ve SON TEMİZLİK
# SQL/Analiz için gerekli ve temiz kolonları seç
df_final = df[[
    'kategori',
    'marka',
    'urun_adi',
    'normal_fiyat',
    'indirimli_fiyat',
    'resim_url',
    'urun_link',
    'urun_tarihi'
]].copy()

# Fiyatı olmayan veya hatalı olan satırları at
df_final = df_final.dropna(subset=['normal_fiyat'])
df_final = df_final[df_final['normal_fiyat'] > 0]

# --- 3. DIŞA AKTARIM (Load - CSV Formatında) ---
print(f"\n💾 Dışa aktarılıyor... Toplam {len(df_final)} temiz satır.")

# CSV olarak dışarı aktar ve Colab'dan indirilmesini sağla
df_final.to_csv(Cikis_Dosya_Adi, index=False)

print(f"✅ İŞLEM BAŞARILI! Veri '{Cikis_Dosya_Adi}' dosyasına kaydedildi.")
print("Lütfen sol menüdeki dosya simgesinden veya otomatik indirme butonuyla dosyayı indirin.")

# İndirme kodunu Colab ortamında etkinleştirelim
from google.colab import files
files.download(Cikis_Dosya_Adi)